# ***Testing Code for image prediction model***

**Libraries**

In [ ]:
# To show the data in table formate.
!pip install tabulate
from tabulate import tabulate

#Libaries for testing car images.
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import load_img
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Libaries for arrays.
import numpy as np

#Libraries for file handling.
import os
import cv2
from glob import glob

#Libraries for visualizing the results.
import matplotlib.pyplot as plt

**Load model**

In [ ]:
model = load_model('D:/car196/car100/alpha0.75_V1.h5')
model.load_weights('D:/car196/car100/weights.05val_acc0.952500.hdf5')
model.save('D:/car196/car100/send_model.h5')
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv1 (Conv2D)               (None, 114, 114, 24)      648       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 114, 114, 24)      96        
_________________________________________________________________
conv1_relu (ReLU)            (None, 114, 114, 24)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 114, 114, 24)      216       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 114, 114, 24)      96        
_________________________________________________________________
conv_dw_1_relu (ReLU)        (None, 114, 114, 24)      0     

In [ ]:
#To get the information of car names.
import csv
with open('D:/car196/car_names.csv') as f:
    output = [s for line in f.readlines() for s in line[:-1].split(',')]
    print(output);

['AM General Hummer SUV 2000', 'Acura RL Sedan 2012', 'Acura TL Sedan 2012', 'Acura ZDX Hatchback 2012', 'Aston Martin V8 Vantage Convertible 2012', 'Aston Martin Virage Convertible 2012', 'Audi A5 Coupe 2012', 'Audi R8 Coupe 2012', 'Audi S4 Sedan 2012', 'Audi S5 Convertible 2012', 'Audi S6 Sedan 2011', 'Audi TT RS Coupe 2012', 'BMW 1 Series Convertible 2012', 'BMW 3 Series Sedan 2012', 'BMW ActiveHybrid 5 Sedan 2012', 'BMW M3 Coupe 2012', 'BMW M5 Sedan 2010', 'BMW M6 Convertible 2010', 'BMW X3 SUV 2012', 'BMW X5 SUV 2007', 'BMW X6 SUV 2012', 'BMW Z4 Convertible 2012', 'Bentley Arnage Sedan 2009', 'Bentley Continental GT Coupe 2012', 'Bentley Mulsanne Sedan 2011', 'Bugatti Veyron 16.4 Convertible 2009', 'Buick Enclave SUV 2012', 'Cadillac CTS-V Sedan 2012', 'Cadillac SRX SUV 2012', 'Chevrolet Avalanche Crew Cab 2012', 'Chevrolet Camaro Convertible 2012', 'Chevrolet Corvette Convertible 2012', 'Chevrolet Corvette ZR1 2012', 'Chevrolet Impala Sedan 2007', 'Chevrolet Malibu Sedan 2007', '

**Testing Part**

In [ ]:
#To test one image.
image = load_img(r'D:\car196\car100\TestImage\Am general.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)

In [ ]:
# To get accuracy for predicting image classwise.
Total_Correctly_Predicted_Images = 0
Correctly_Predicted_Images = 0

train_path = [x[0] for x in os.walk('D:/car196/car100/testing')]
train_path=sorted(train_path)
data=[]

for i in range(1,len(train_path)):
    files = os.listdir(str(train_path[i]))
    for j in range(0,len(files)):
        path = os.path.join(str(train_path[i]),str(files[j]))
        
        image = load_img(path, target_size=(227, 227))
        image = img_to_array(image)
        image = image.reshape((1, 227, 227, 3))
        image = preprocess_input(image)
        yhat = model.predict(image)
        
        if np.argmax(np.array(yhat[0]))==i-1:
          Correctly_Predicted_Images+=1
    temp  = train_path[i].split('/')

    data.append([temp[-1],cnt/len(files)])
    Total_Correctly_Predicted_Images += Correctly_Predicted_Images
    Correctly_Predicted_Images = 0
    
print(tabulate(data,headers=("Car","Accuracy"),tablefmt='fancy_grid',showindex=True,numalign="left"))

╒════╤════════════════════════════════════════════════════════════╤════════════╕
│    │ Car                                                        │ Accuracy   │
╞════╪════════════════════════════════════════════════════════════╪════════════╡
│ 0  │ testing\AM General Hummer SUV 2000                         │ 1          │
├────┼────────────────────────────────────────────────────────────┼────────────┤
│ 1  │ testing\Acura RL Sedan 2012                                │ 0.8        │
├────┼────────────────────────────────────────────────────────────┼────────────┤
│ 2  │ testing\Acura TL Sedan 2012                                │ 1          │
├────┼────────────────────────────────────────────────────────────┼────────────┤
│ 3  │ testing\Acura ZDX Hatchback 2012                           │ 0.85       │
├────┼────────────────────────────────────────────────────────────┼────────────┤
│ 4  │ testing\Aston Martin V8 Vantage Convertible 2012           │ 0.85       │
├────┼──────────────────────

In [ ]:
# To get overall accuracy for the model.

test_datagen = ImageDataGenerator(preprocessing_function= preprocess_input)
test_set = test_datagen.flow_from_directory('D:/car196/car100/testing',
                                            target_size = (227, 227),
                                            batch_size = 64,
                                            class_mode = 'categorical',
                                            shuffle=True)

model.evaluate(test_set)

Found 2000 images belonging to 100 classes.
32/32 [==============================] - 3s 83ms/step - loss: 0.1989 - accuracy: 0.9600


[0.19886669516563416, 0.9599999785423279]

**BlackBox testing**

In [ ]:
#invalid class. 
image = load_img(r'D:\car196\car100\TestImage\dog.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(np.argmax(yhat))
if(compare[9]<0.75):
    print("Invalid")

In [ ]:
#invalid class
image = load_img(r'D:\car196\car100\TestImage\hills_1.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(np.argmax(yhat))
if(yhat[0][compare[9]]<0.75):
    print("Invalid")

In [ ]:
#invalid class 
image = load_img(r'D:\car196\car100\TestImage\river.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(np.argmax(yhat))
if(yhat[0][compare[9]]<0.75):
    print("Invalid")

In [ ]:
#invalid class 
image = load_img(r'D:\car196\car100\TestImage\tree.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(np.argmax(yhat))
if(yhat[0][compare[9]]<0.75):
    print("Invalid")

In [ ]:
#boundary case with unavailable class
image = load_img(r'D:\car196\car100\TestImage\wrong.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(output[np.argmax(yhat)])
if(yhat[0][compare[9]]<0.75):
    print("Invalid")
    

In [ ]:
#boundary case with valid class and noise
image = load_img(r'D:\car196\car100\TestImage\carwithnoise.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(output[np.argmax(yhat)])
if(yhat[0][compare[9]]<0.75):
    print("Invalid")
    

In [ ]:
#boundary case with valid class and noise
image = load_img(r'D:\car196\car100\TestImage\valid.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(output[np.argmax(yhat)])
if(yhat[0][compare[9]]<0.75):
    print("Invalid")
    

In [ ]:
#boundary case with valid class and noise
image = load_img(r'D:\car196\car100\TestImage\valid2.jpg', target_size=(227, 227))
image = img_to_array(image)
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
image=preprocess_input(image)
yhat = model.predict(image)
compare = np.argsort(yhat)[0][90 : 100]
for x in compare:
    print(yhat[0][x])
print(output[np.argmax(yhat)])
if(yhat[0][compare[9]]<0.75):
    print("Invalid")
    